In [1]:
# define a model using pytorch
# a pretrained model adding layers to classify the 4 classes of the images
import torch.nn as nn
from torchvision import models
from torchvision.models import squeezenet1_1
from torchvision.models.squeezenet import SqueezeNet1_1_Weights



class MIDmodel(nn.Module):
    def __init__(self):
        super(MIDmodel, self).__init__()
        self.pretrained = squeezenet1_1()
        self.fc1 = nn.Linear(1000, 128)
        self.fc2 = nn.Linear(128, 4)
        self.dropout = nn.Dropout(0.7)

    def forward(self, x):
        x = self.pretrained(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [2]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, root_dir, patients_ids, patients_df, transform=None):
        # select the rows of the dataframe that correspond to the patients in the list
        self.patients_df = patients_df[patients_df["patient_id"].isin(patients_ids)]
        self.root_dir = root_dir
        self.idxs = set()
        if transform:
            self.transform = transform
            # duplicate the patients_df, so the network will see each image twice, one standard and one transformed
            # self.patients_df = pd.concat([self.patients_df] * 2, ignore_index=True)
        else:
            self.transform = None
               
            
    def __len__(self):
        return len(self.patients_df)

    def __getitem__(self, idx):
        # get idx row of the dataframe
        row = self.patients_df.iloc[idx]
        # path = root_dir/patientid_examid_spot_framenumber_score.png
        path = os.path.join(self.root_dir, row["patient_id"] + "_" + row["exam_id"] + "_" + row["spot"] + "_" + row["frame_number"] + "_" + row["score"] + ".png")
        image = Image.open(path)
        label = row["score"]
        # if the image is grayscale, convert it to RGB (used to test with the smaller dataset)
        # if image.mode != "RGB":
        #     image = image.convert("RGB")
               
        if self.transform:
            image = self.transform(image)
            # if path not in self.idxs: # if it's the first time we see the image, add its path to the set
            #     self.idxs.add(path)
            # else: # if it's the second time we see the image, apply the transform and remove its path from the set 
            #     image = self.transform(image)
            #     self.idxs.remove(path)
        # in any case, transform to tensor
        image = transforms.ToTensor()(image)
        image = transforms.Resize((224, 224))(image)
        # Finally the values are first rescaled to [0.0, 1.0] and then normalized using mean=[0.485, 0.456, 0.406] and std=[0.229, 0.224, 0.225].
        image = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(image)
        return image, int(label)

In [3]:
import pandas as pd
import torch
import glob
data_dir = "images/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
images_paths = glob.glob(f"{data_dir}*.png", recursive=True)
# images are named as: patientid_examid_spotnumber_framenumber_score.png
# create a dataframe with the data removing "images/"
images_df = pd.DataFrame([path[len(data_dir):-4].split("_") for path in images_paths], columns=["patient_id", "exam_id", "spot", "frame_number", "score"])
images_df["score"] = images_df["score"].astype(str)
images_df["frame_number"] = images_df["frame_number"].astype(str)
images_df["spot"] = images_df["spot"].astype(str)
images_df["patient_id"] = images_df["patient_id"].astype(str)
images_df["exam_id"] = images_df["exam_id"].astype(str)
patients_ids = set(images_df["patient_id"])
patients_ids = list(patients_ids)

# select the 8 patients for training based on the most balanced distribution of the scores
import itertools
combs = list(itertools.combinations(patients_ids, 8))
stds = []
for i, c in enumerate(combs):
    stds.append((images_df[images_df["patient_id"].isin(c)].groupby("score").count()["patient_id"].std(), i))

train_patients = [x for x in combs[min(stds)[1]]]
print(train_patients)
print(images_df[images_df["patient_id"].isin(train_patients)].groupby("score").count()["patient_id"])
test_patients = [x for x in patients_ids if x not in train_patients]
print(test_patients)
print(images_df[images_df["patient_id"].isin(test_patients)].groupby("score").count()["patient_id"])


['1051', '1017', '1050', '1052', '1069', '1048', '1067', '1047']
score
0    7888
1    7540
2    7189
3    7592
Name: patient_id, dtype: int64
['1066', '1068', '1045']
score
0    6625
1    1721
2    5535
3    3631
Name: patient_id, dtype: int64


transformations from the paper Deep Learning for Classification and Localization of COVID-19 Markers in Point-of-Care Lung Ultrasound
were each activated on the image-label pair with a probability of 0.33. The set of augmentation functions, each applied with a randomly sampled strength bounded by a set maximum, consists of:
- affine transformations (translation (max. ±15%), rotation (max. ±15◦), scaling (max. ±45%), and shearing (max. ±4.5◦))
- multiplication with a constant (max. ±45%)
- Gaussian blurring (σmax = 3/4 )
- contrast distortion (max. ±45%)
- horizontal flipping (p = 0.5)
- additive white Gaussian noise (σmax = 0.015)

In [4]:
transformations = transforms.Compose([
    transforms.RandomApply([transforms.RandomAffine(degrees=15, translate=(0.15, 0.15), scale=(0.55, 1.45), shear=4.5)], p=1/3),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.45, contrast=0.45, saturation=0.45, hue=0.45)], p=1/3),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=3, sigma=(0.75))], p=1/3),
    transforms.RandomApply([transforms.RandomHorizontalFlip(p=0.5)], p=1/3)
    ])

In [5]:
from torch import optim
from tqdm import tqdm

def train_model(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    for inputs, labels in tqdm(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects/ len(dataloader.dataset)
    return epoch_loss, epoch_acc

def test_model(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects / len(dataloader.dataset)
    return epoch_loss, epoch_acc


In [6]:
data_dir = "images/"
num_epochs = 10
learning_rate = 0.00001
batch_size = 8

model = MIDmodel()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


train_dataset = ImageDataset(data_dir, train_patients, images_df, transformations)
test_dataset = ImageDataset(data_dir, test_patients, images_df)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [21]:
results_train = []
results_test = []

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    train_loss, train_acc = train_model(model, train_dataloader, optimizer, criterion, device)
    results_train.append([train_loss, train_acc])
    print(f"Training Loss: {train_loss:.4f} Training Accuracy: {train_acc:.4f}")
    test_loss, test_acc = test_model(model, test_dataloader, criterion, device)
    results_test.append([test_loss, test_acc])
    print(f"Test Loss: {test_loss:.4f} Test Accuracy: {test_acc:.4f}")
    if epoch % 5 == 0:
        torch.save(model.state_dict(), f"MIDmodel_{epoch}.pt")
    if epoch % 2 == 1:
        learning_rate = learning_rate/2
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print("Train: ", results_train)
print("Test: ", results_test)

# save the model
torch.save(model.state_dict(), f"MIDmodel.pt")

Epoch 0


100%|██████████| 3777/3777 [17:01<00:00,  3.70it/s]


Training Loss: 1.4453 Training Accuracy: 0.2713


100%|██████████| 2189/2189 [04:26<00:00,  8.21it/s]


Test Loss: 1.3651 Test Accuracy: 0.3936
Epoch 1


...

Test Loss: 1.4515 Test Accuracy: 0.4854
Epoch 9


100%|██████████| 3777/3777 [16:44<00:00,  3.76it/s]


Training Loss: 0.5772 Training Accuracy: 0.7737


100%|██████████| 2189/2189 [03:58<00:00,  9.18it/s]

Test Loss: 1.5062 Test Accuracy: 0.4978
Train:  [[1.4453151551324965, tensor(0.2713, device='cuda:0')], [1.278736829047383, tensor(0.4056, device='cuda:0')], [1.0870531683799356, tensor(0.5218, device='cuda:0')], [0.923765770988631, tensor(0.6007, device='cuda:0')], [0.8183876516514437, tensor(0.6576, device='cuda:0')], [0.7402640066483773, tensor(0.6972, device='cuda:0')], [0.6809365507032136, tensor(0.7247, device='cuda:0')], [0.6463492192383963, tensor(0.7430, device='cuda:0')], [0.6037752303301651, tensor(0.7579, device='cuda:0')], [0.5771569872453134, tensor(0.7737, device='cuda:0')]]
Test:  [[1.3650751340982847, tensor(0.3936, device='cuda:0')], [1.3009044412419697, tensor(0.4223, device='cuda:0')], [1.507390807594214, tensor(0.3301, device='cuda:0')], [1.6292844691393364, tensor(0.4055, device='cuda:0')], [1.5698632373725132, tensor(0.4069, device='cuda:0')], [1.5488062066295345, tensor(0.4343, device='cuda:0')], [1.4955579021049161, tensor(0.4344, device='cuda:0')], [1.75228601

Classifier to predict if the confidence is high enough, created on the entire dataset

In [36]:
# load the last model
model = MIDmodel()
model.load_state_dict(torch.load("MIDmodel.pt"))
model.to(device)
model.eval()

correct_preds = pd.DataFrame(columns=["true", "predicted", "sm0", "sm1", "sm2", "sm3"])
wrong_preds = pd.DataFrame(columns=["true", "predicted", "sm0", "sm1", "sm2", "sm3"])

entire_dataset = ImageDataset(data_dir, patients_ids, images_df)

for i in tqdm(range(len(entire_dataset))):
    img, label = entire_dataset[i]
    img = img.unsqueeze(0)
    img = img.to(device)
    output = model(img)
    sm = nn.Softmax(dim=1)
    sm = sm(output)
    sm = sm.cpu().detach().numpy()
    sm = sm[0]
    if torch.argmax(output) == label:
        correct_preds = pd.concat([correct_preds, pd.DataFrame([[label, torch.argmax(output).cpu().detach().numpy(), sm[0], sm[1], sm[2], sm[3]]], columns=["true", "predicted", "sm0", "sm1", "sm2", "sm3"])])
    else:
        wrong_preds = pd.concat([wrong_preds, pd.DataFrame([[label, torch.argmax(output).cpu().detach().numpy(), sm[0], sm[1], sm[2], sm[3]]], columns=["true", "predicted", "sm0", "sm1", "sm2", "sm3"])])

correct_preds.to_csv("correct_preds.csv", index=False)
wrong_preds.to_csv("wrong_preds.csv", index=False)


100%|██████████| 2760/2760 [00:48<00:00, 56.65it/s]


In [9]:


class MIDbinary(nn.Module):
    def __init__(self):
        super(MIDbinary, self).__init__()
        self.fc1 = nn.Linear(4, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 2)
        self.dropout = nn.Dropout(0.2)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        x = self.softmax(x)
        return x
    
# BinaryDataset
class BinaryDataset(Dataset):
    def __init__(self, correct_preds, wrong_preds):
        # add a clumn to correct_preds and wrong_preds that says if it's correct or not
        correct_preds["correct"] = 1
        wrong_preds["correct"] = 0
        self.values = pd.concat([correct_preds, wrong_preds])

    def __len__(self):
        return len(self.values)

    def __getitem__(self, idx):
        label = self.values.iloc[idx]["correct"]
        sms = self.values.iloc[idx][["sm0", "sm1", "sm2", "sm3"]]
        # print(list(sms.values))
        sms = torch.tensor(list(sms.values), dtype=torch.float32)
        return sms, int(label)

# train the binary model
binary_model = MIDbinary()
binary_model = binary_model.to(device)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.00001
optimizer = optim.Adam(binary_model.parameters(), lr=learning_rate)

correct_preds = pd.read_csv("correct_preds.csv")
wrong_preds = pd.read_csv("wrong_preds.csv")

# divide train and test data
train_size = int(0.8 * (len(correct_preds)+len(wrong_preds)))
test_size = len(correct_preds)+len(wrong_preds) - train_size
from torch.utils.data import random_split
train_dataset, test_dataset = random_split(BinaryDataset(correct_preds, wrong_preds), [train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

results_train = []
results_test = []

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    train_loss, train_acc = train_model(binary_model, train_dataloader, optimizer, criterion, device)
    results_train.append([train_loss, train_acc.item()])
    print(f"Training Loss: {train_loss:.4f} Training Accuracy: {train_acc:.4f}")
    test_loss, test_acc = test_model(binary_model, test_dataloader, criterion, device)
    results_test.append([test_loss, test_acc.item()])
    print(f"Test Loss: {test_loss:.4f} Test Accuracy: {test_acc:.4f}")
    if epoch % 5 == 0:
        torch.save(binary_model.state_dict(), f"binary_model_{epoch}.pt")
    if epoch % 2 == 1:
        learning_rate = learning_rate/2
        optimizer = optim.Adam(binary_model.parameters(), lr=learning_rate)

print("Train: ", results_train)
print("Test: ", results_test)


Epoch 0


100%|██████████| 276/276 [00:01<00:00, 181.32it/s]


Training Loss: 0.7218 Training Accuracy: 0.1051


100%|██████████| 69/69 [00:00<00:00, 265.60it/s]


Test Loss: 0.7160 Test Accuracy: 0.0942
Epoch 1


...

Test Loss: 0.6802 Test Accuracy: 0.9058
Epoch 9


100%|██████████| 276/276 [00:01<00:00, 172.37it/s]


Training Loss: 0.6806 Training Accuracy: 0.7310


100%|██████████| 69/69 [00:00<00:00, 256.09it/s]

Test Loss: 0.6794 Test Accuracy: 0.9058
Train:  [[0.7217894384394521, 0.10507246851921082], [0.7099223804214726, 0.16530796885490417], [0.7007769164831742, 0.3134058117866516], [0.6942635046831076, 0.46557971835136414], [0.6894621818825819, 0.5765398740768433], [0.6870639650286108, 0.626358687877655], [0.6843310277099195, 0.6653079986572266], [0.6826637391594873, 0.7155797481536865], [0.6809807108796161, 0.7250906229019165], [0.6806028567362523, 0.73097825050354]]
Test:  [[0.7159783365069956, 0.09420289844274521], [0.7037074565887451, 0.09420289844274521], [0.6974990661593451, 0.5507246255874634], [0.691044807434082, 0.6086956858634949], [0.6877917630085046, 0.6123188734054565], [0.6844672284264496, 0.6177536249160767], [0.6827566148578257, 0.6123188734054565], [0.6810366044873777, 0.820652186870575], [0.6801887266877769, 0.9057971239089966], [0.6793522307838219, 0.9057971239089966]]


t-SNE to visualize only the correct prediction (but still in both train and test)

In [ ]:
# impiort TNSE
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

